In [2]:
!pip install -q transformers datasets accelerate evaluate huggingface_hub torch torchvision tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [14]:
import torch
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))
model = model.to("cuda")


True Tesla T4


In [3]:
from huggingface_hub import notebook_login
notebook_login()  # ⬅️ paste your Hugging Face token (from https://huggingface.co/settings/tokens)


In [4]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

MODEL_NAME = "facebook/deit-tiny-patch16-224"

processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForImageClassification.from_pretrained(MODEL_NAME)

print("✅ Base model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


pytorch_model.bin:   0%|          | 0.00/23.0M [00:00<?, ?B/s]

✅ Base model loaded successfully!


In [4]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
Saving My_Resume (7).pdf to My_Resume (7).pdf


In [5]:
!kaggle datasets list -s deepfashion2

ref                                                 title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ---------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
thusharanair/deepfashion2-original-with-dataframes  DeepFashion2 Original (with Dataframes)  15995030274  2024-01-10 10:02:50.423000           1891          6  0.875            
hassam361/deepfashion2                              deepfashion2                             15757939282  2020-11-11 11:44:08.133000            646          7  0.125            
thusharanair/deepfashion2-256x256                   DeepFashion2: Resized to 256 x 256        7500182549  2024-01-18 15:16:52.170000            200          2  0.875            
thaiminhpv/deepfashion2-deephashing                 DeepFashion2-DeepHashing                 16031464653  2023

In [6]:
from datasets import load_dataset

# Small dataset of fashion product images (public & lightweight)
dataset = load_dataset("ashraq/fashion-product-images-small", split="train[:500]")
print(dataset)

Dataset({
    features: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName', 'image'],
    num_rows: 500
})


In [7]:
import pandas as pd

records = []
for item in dataset:
    records.append({
        "image": item["image"].filename if hasattr(item["image"], "filename") else None,
        "type": item.get("articleType", "unknown"),
        "color": item.get("baseColour", "unknown"),
        "pattern": item.get("usage", "plain")  # 'usage' is used as a stand-in for pattern
    })

df = pd.DataFrame(records)
df.head()


,image,type,color,pattern
0,,Shirts,Navy Blue,Casual
1,,Jeans,Blue,Casual
2,None,Watches,Silver,Casual
3,,Track Pants,Black,Casual
4,None,Tshirts,Grey,Casual


In [9]:
import os
from PIL import Image
import pandas as pd

# Create folder to store images
os.makedirs("data/images", exist_ok=True)

records = []
for i, item in enumerate(dataset):
    # Save image as JPG
    img_path = f"data/images/{i:04d}.jpg"
    item["image"].save(img_path)

    # Record metadata + absolute path
    records.append({
        "image": os.path.abspath(img_path),
        "type": item.get("articleType", "unknown"),
        "color": item.get("baseColour", "unknown"),
        "pattern": item.get("usage", "plain")
    })

# Build dataframe and save CSV
df = pd.DataFrame(records)
csv_path = "data/labels_train.csv"
df.to_csv(csv_path, index=False)

print(f" Saved {len(df)} images to 'data/images/'")
print(f" Labels file: {os.path.abspath(csv_path)}")
df.head()


 Saved 500 images to 'data/images/'
 Labels file: /content/data/labels_train.csv


,image,type,color,pattern
0,/content/data/images/0000.jpg,Shirts,Navy Blue,Casual
1,/content/data/images/0001.jpg,Jeans,Blue,Casual
2,/content/data/images/0002.jpg,Watches,Silver,Casual
3,/content/data/images/0003.jpg,Track Pants,Black,Casual
4,/content/data/images/0004.jpg,Tshirts,Grey,Casual


In [10]:
from PIL import Image
import pandas as pd

df = pd.read_csv("data/labels_train.csv")
print(df.head())

# Open one image
img = Image.open(df.iloc[0]["image"])
img.show()


                           image         type      color pattern
0  /content/data/images/0000.jpg       Shirts  Navy Blue  Casual
1  /content/data/images/0001.jpg        Jeans       Blue  Casual
2  /content/data/images/0002.jpg      Watches     Silver  Casual
3  /content/data/images/0003.jpg  Track Pants      Black  Casual
4  /content/data/images/0004.jpg      Tshirts       Grey  Casual


In [11]:
# Load dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd

df = pd.read_csv("data/labels_train.csv")

type_encoder = LabelEncoder().fit(df["type"])
color_encoder = LabelEncoder().fit(df["color"])
pattern_encoder = LabelEncoder().fit(df["pattern"])

df["type_id"] = type_encoder.transform(df["type"])
df["color_id"] = color_encoder.transform(df["color"])
df["pattern_id"] = pattern_encoder.transform(df["pattern"])

print("✅ Label encoding done")
print("Types:", len(type_encoder.classes_), "| Colors:", len(color_encoder.classes_), "| Patterns:", len(pattern_encoder.classes_))
df.head()


✅ Label encoding done
Types: 70 | Colors: 37 | Patterns: 4


,image,type,color,pattern,type_id,color_id,pattern_id
0,/content/data/images/0000.jpg,Shirts,Navy Blue,Casual,46,19,0
1,/content/data/images/0001.jpg,Jeans,Blue,Casual,29,2,0
2,/content/data/images/0002.jpg,Watches,Silver,Casual,69,30,0
3,/content/data/images/0003.jpg,Track Pants,Black,Casual,60,1,0
4,/content/data/images/0004.jpg,Tshirts,Grey,Casual,65,12,0


In [12]:
# Preprocess subset dataset
from datasets import Dataset
from transformers import AutoImageProcessor
from PIL import Image
import torch

MODEL_NAME = "facebook/deit-tiny-patch16-224"
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")
    example["pixel_values"] = processor(images=image, return_tensors="pt")["pixel_values"][0]
    example["labels"] = torch.tensor([
        example["type_id"],
        example["color_id"],
        example["pattern_id"]
    ], dtype=torch.long)
    return example

dataset = Dataset.from_pandas(df)
dataset = dataset.map(preprocess)
dataset.set_format(type="torch", columns=["pixel_values", "labels"])
print(dataset)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'type', 'color', 'pattern', 'type_id', 'color_id', 'pattern_id', 'pixel_values', 'labels'],
    num_rows: 500
})


In [15]:
# Define training processes ViT model with attirbutes
from transformers import AutoModel
import torch.nn as nn

class MultiHeadDeiT(nn.Module):
    def __init__(self, base_model_name, n_types, n_colors, n_patterns):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_name)
        hidden = self.backbone.config.hidden_size
        self.type_head = nn.Linear(hidden, n_types)
        self.color_head = nn.Linear(hidden, n_colors)
        self.pattern_head = nn.Linear(hidden, n_patterns)

    def forward(self, pixel_values, labels=None):
        outputs = self.backbone(pixel_values)
        pooled = outputs.last_hidden_state[:, 0]  # CLS token
        type_logits = self.type_head(pooled)
        color_logits = self.color_head(pooled)
        pattern_logits = self.pattern_head(pooled)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            t_loss = loss_fn(type_logits, labels[:, 0])
            c_loss = loss_fn(color_logits, labels[:, 1])
            p_loss = loss_fn(pattern_logits, labels[:, 2])
            loss = t_loss + c_loss + p_loss

        return {"loss": loss, "logits": (type_logits, color_logits, pattern_logits)}

model = MultiHeadDeiT(
    base_model_name=MODEL_NAME,
    n_types=len(type_encoder.classes_),
    n_colors=len(color_encoder.classes_),
    n_patterns=len(pattern_encoder.classes_)
)


Some weights of ViTModel were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import os
os.environ["WANDB_MODE"] = "disabled"


In [20]:
# Training with arguments
from transformers import Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset,
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
10,8.552100
20,7.526300
30,7.437600
40,6.773700
50,6.742100
60,6.263600
70,6.138200
80,5.464800
90,5.309300
100,5.004900


TrainOutput(global_step=189, training_loss=5.433518344132358, metrics={'train_runtime': 26.8014, 'train_samples_per_second': 55.967, 'train_steps_per_second': 7.052, 'total_flos': 0.0, 'train_loss': 5.433518344132358, 'epoch': 3.0})

In [21]:
# Save model in run time
trainer.save_model("./final_model")
processor.save_pretrained("./final_model")

['./final_model/preprocessor_config.json']

In [22]:
# Test model with the first image
from PIL import Image
import torch

img_path = df.iloc[0]["image"]
image = Image.open(img_path).convert("RGB")

inputs = processor(images=image, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = model(**inputs)

type_logits, color_logits, pattern_logits = outputs["logits"]
pred_type = type_encoder.classes_[type_logits.argmax().item()]
pred_color = color_encoder.classes_[color_logits.argmax().item()]
pred_pattern = pattern_encoder.classes_[pattern_logits.argmax().item()]

print(f" Type: {pred_type}")
print(f" Color: {pred_color}")
print(f" Pattern: {pred_pattern}")


 Type: Shirts
 Color: Blue
 Pattern: Casual


In [26]:
# Create hugging face repo
from huggingface_hub import create_repo

create_repo(
    repo_id="changju784/clothing-app-fine-tune",
    repo_type="model",
    private=False
)

RepoUrl('https://huggingface.co/changju784/clothing-app-fine-tune', endpoint='https://huggingface.co', repo_type='model', repo_id='changju784/clothing-app-fine-tune')

In [27]:
# Push model to hugging face
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="./final_model",
    repo_id="changju784/clothing-app-fine-tune",
    repo_type="model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...l_model/model.safetensors:   2%|2         |  550kB / 22.4MB            

  ...l_model/training_args.bin:   2%|2         |   142B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/changju784/clothing-app-fine-tune/commit/df4f7b2f5fdef2249642febbb042ce5561ac332e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='df4f7b2f5fdef2249642febbb042ce5561ac332e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/changju784/clothing-app-fine-tune', endpoint='https://huggingface.co', repo_type='model', repo_id='changju784/clothing-app-fine-tune'), pr_revision=None, pr_num=None)

In [28]:
# Push to github
!git config --global user.name "changju784"
!git config --global user.email "wjddlsdl784@gmail.com"
!git clone https://github.com/changju784/clothing-app-fine-tune.git
!cp -r final_model/* clothing-app-fine-tune/
!cd clothing-app-fine-tune && git add . && git commit -m "Add fine-tuned DeiT model and README" && git push

Cloning into 'clothing-app-fine-tune'...
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ae996e7f6731.(none)')
